In [2]:
import pandas as pd
"""
Right now we are downloading data from 
https://fragilestatesindex.org/
Then processing it for move up to red
We need to automate this process for use in our new green to red pipeline(WIP)
On a monthly basis we need to do a data pull and get any new data
If we put it in a dash app we can use POSIT CONNECT to keep the script persisting
"""
df = pd.read_excel("Rankings.xlsx")

cols = []
for col in df.columns:
    if "Unnamed" in col:
        cols.append(df[col].iloc[0])
    else:
        cols.append(col)
        
df.columns = cols
df = df[1:]

df = df[["Country", "Total"]]
df.columns = ["Country", "FSI Total"]

df["FSI Total Normalized"] = df["FSI Total"].apply(lambda x: (x-min(df["FSI Total"]))/(max(df["FSI Total"])-min(df["FSI Total"])))
df["FSI Total Normalized x4"] = df["FSI Total Normalized"]*4

In [3]:
df

,Country,FSI Total,FSI Total Normalized,FSI Total Normalized x4
1,Somalia,111.3,1.000000,4.000000
2,Sudan,109.3,0.979716,3.918864
3,South Sudan,109.0,0.976673,3.906694
4,Syria,108.1,0.967546,3.870183
5,Congo Democratic Republic,106.7,0.953347,3.813387
...,...,...,...,...
175,Denmark,15.9,0.032454,0.129817
176,New Zealand,15.9,0.032454,0.129817
177,Iceland,15.2,0.025355,0.101420
178,Finland,14.3,0.016227,0.064909


In [4]:
df.to_excel("FSI_output.xlsx")

In [5]:
import dash
from dash import Dash, dcc, html
import matplotlib.pyplot as plt
import plotly.express as px

In [6]:
column_names = df.columns
print("Column names", column_names.tolist())

Column names ['Country', 'FSI Total', 'FSI Total Normalized', 'FSI Total Normalized x4']


In [7]:
fig = px.bar(
    df,
    x="FSI Total",
    y="Country",
    orientation="h",
    title="Fragile State Index per Country",
    labels={"FSI Total": "FSI Total Score", "Country": "Countries"},
)


In [8]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Green to Red Pipeline - Fragile State Index"),
    dcc.Graph(figure=fig),
])

In [9]:
if __name__== "__main__":
    app.run_server(debug=True)

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# Path to your ChromeDriver
driver_path = "path/to/chromedriver"

# Set up Selenium WebDriver
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

url = "https://fragilestatesindex.org/excel/"
driver.get(url)

# Wait for the page to load (adjust time if necessary)
driver.implicitly_wait(10)

# Extract links
links = []
for link in driver.find_elements(By.TAG_NAME, "a"):
    href = link.get_attribute("href")
    if href and (".xlsx" in href or ".xls" in href):
        links.append(href)

print("Excel Links Found:", links)

driver.quit()

ModuleNotFoundError: No module named 'selenium'